In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader

### Create dataset

In [3]:
data_path = 'ascii_data/cleaned_ascii_art_data.json'
# the path leads to a json file that was saved from a pandas series
df = pd.read_json(data_path, typ='series')

In [4]:
print(df[0])

                                                            
                                                            
                                                            
                                                            
                                                            
                                                            
                    ,__    _,            ___                
                   '.`\ /`|     _.-"```   `'.               
                    ; |  /   .'             `}              
                    _\|\/_.-'                 }             
               _.-"a                 {        }             
            .-`  __    /._          {         }\            
           '--"`  `""`   `\   ;    {         } \            
                          |   } __ _\       }\  \           
                         |  /;`   / :.   }`  \  \           
                        | | | .-' /  / /     '. '._         
             jgs    .'__

In [23]:
class AsciiArtDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        art = self.data[index]
        lines = art.split('\r\n')
        ascii_tensor = torch.zeros((len(lines), len(lines[0])), dtype=torch.float32)
        for i, line in enumerate(lines):
            for j, char in enumerate(line):
                ascii_tensor[i][j] = (ord(char) - 32) / 95  # Normalize the ASCII values to [0, 1]
        return ascii_tensor

In [6]:
ascii_data = AsciiArtDataset(df)
dataloader = DataLoader(ascii_data, batch_size=4, shuffle=True)

In [7]:
# Get a batch of data
dataiter = next(iter(dataloader))
print(dataiter[0])

                             ,   .-'"'=;_  ,                
                             |\.'-~`-.`-`;/|                
                             \.` '.'~-.` './                
                             (\`,__=-'__,'/)                
                         _.-'-.( d\_/b ).-'-._              
                      /'.-'   ' .---. '   '-.`\             
                   /'  .' (=    (_)    =) '.  `\            
                 /'  .',  `-.__.-.__.-'  ,'.  `\            
                (     .'.   V       V  ; '.     )           
                (    |::  `-,__.-.__,-'  ::|    )           
                |   /|`:.               .:'|\   |           
                |  / | `:.              :' |`\  |           
                | |  (  :.             .:  )  | |           
                | |   ( `:.            :' )   | |           
                | |    \ :.           .: /    | |           
                | |     \`:.         .:'/     | |           
                ) (     

In [8]:
UNIQUE_CHARS = set()
for i in range(len(df)):
    for char in df[i]:
        UNIQUE_CHARS.add(char)

UNIQUE_CHARS = list(UNIQUE_CHARS)
len(UNIQUE_CHARS)

111

## Modeling

In [32]:
class VAE(nn.Module):
    def __init__(self, input_size, latent_dim):
        super(VAE, self).__init__()
        self.input_size = input_size
        self.latent_dim = latent_dim

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64 * 15 * 6, 128),
            nn.ReLU(),
            nn.Linear(128, 2 * latent_dim)
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64 * 15 * 6),
            nn.ReLU(),
            nn.Unflatten(1, (64, 15, 6)),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def encode(self, x):
        x = self.encoder(x)
        mu, log_var = x.chunk(2, dim=-1)
        return mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        return self.decoder(z)

    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_recon = self.decode(z)
        return x_recon, mu, log_var

def loss_function(x_recon, x, mu, log_var):
    recon_loss = nn.BCELoss(reduction='sum')(x_recon, x)
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return recon_loss + kl_loss

def train(model, dataloader, optimizer, device, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        for batch in dataloader:
            batch = batch.to(device).unsqueeze(1).float()  # Move tensor to device, add channel dimension, and convert to float
            optimizer.zero_grad()
            x_recon, mu, log_var = model(batch)
            loss = loss_function(x_recon, batch, mu, log_var)
            loss.backward()
            optimizer.step()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

def generate_samples(model, num_samples, device):
    with torch.no_grad():
        z = torch.randn(num_samples, model.latent_dim).to(device)
        samples = model.decode(z)
    return samples

In [33]:
# Set up the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set up the dataset and dataloader
dataset = AsciiArtDataset(df)  # Assuming you have already defined your dataset
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Set up the model, optimizer, and hyperparameters
input_size = (60, 24)
latent_dim = 10
model = VAE(input_size, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100

# Train the model
train(model, dataloader, optimizer, device, num_epochs)

RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[1, 32, 24, 60] to have 1 channels, but got 32 channels instead

In [ ]:
# Generate new ASCII art samples
num_samples = 10
generated_samples = generate_samples(model, num_samples, device)

# Convert generated samples to ASCII art strings
generated_art = []
for sample in generated_samples:
    sample = sample.squeeze().cpu().numpy()
    ascii_art = ""
    for row in sample:
        ascii_row = "".join([chr(int(char * 95) + 32) for char in row])
        ascii_art += ascii_row + "\n"
    generated_art.append(ascii_art)

# Print or save the generated ASCII art
for art in generated_art:
    print(art)